In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Laptop crashed while training the images using Graph Neural Networks. Hence this simple NLP approach, but efficient. 

In [4]:
#!pip install fuzzywuzzy
#!pip install python-Levenshtein
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download en_core_web_md

import pandas as pd
import numpy as np
from datetime import datetime
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.parsing.preprocessing import STOPWORDS
from fuzzywuzzy import process
import en_core_web_md
from sklearn.metrics.pairwise import cosine_similarity

start = datetime.now()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# read outfit file
outfit = pd.read_csv('https://dso-560-nlp-text-analytics.s3.amazonaws.com/outfit_combinations.csv')
outfit

,outfit_id,product_id,outfit_item_type,brand,product_full_name
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory1,kate spade new york,medium margaux leather satchel
3,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2ZFDYRYY5TRQZJTBD,shoe,Tory Burch,Penelope Mid Cap Toe Pump
4,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
...,...,...,...,...,...
5286,01E6MC52DNZE5AZ9MPZ2CYCXW9,01E5ZYHZA7186DVWEJ99Q4D2PM,accessory2,Sam Edelman,65mm Gradient Oversize Square Sunglasses
5287,01E6MC52DPCTAAQVYNSKA7PER7,01E2P0SJSKFKNQJ5SVQ8MD1JZT,shoe,DR. MARTENS,Fenimore Triple Buckle Boot
5288,01E6MC52DPCTAAQVYNSKA7PER7,01E4RW25Y8ZF6WKZRE50Y6SKH5,onepiece,Nili Lotan,Pamela Dress
5289,01E6MC52DPCTAAQVYNSKA7PER7,01E5ZS3R9JD696YWGK9NSG56E1,accessory1,Mansur Gavriel,Leather Circle Crossbody Bag


In [7]:
outfit.outfit_item_type.value_counts()

shoe          1149
accessory1    1064
accessory2     978
top            950
bottom         928
onepiece       221
accessory3       1
Name: outfit_item_type, dtype: int64

In [8]:
# replace accessory1/2/3 with accessory
outfit['outfit_item_type'] = outfit.outfit_item_type.str.replace(r'(accessory\d)', r'accessory')

In [10]:
# read full data file
full_data = pd.read_csv('https://dso-560-nlp-text-analytics.s3.amazonaws.com/Full+data.csv')
full_data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,product_id,brand,mpn,name,description,brand_category,created_at,updated_at,deleted_at,brand_canonical_url,details,labels,tsv,bc_product_id,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220,Unnamed: 221,Unnamed: 222,Unnamed: 223,Unnamed: 224,Unnamed: 225,Unnamed: 226,Unnamed: 227,Unnamed: 228,Unnamed: 229,Unnamed: 230,Unnamed: 231,Unnamed: 232,Unnamed: 233,Unnamed: 234,Unnamed: 235,Unnamed: 236,Unnamed: 237,Unnamed: 238,Unnamed: 239,Unnamed: 240,Unnamed: 241,Unnamed: 242,Unnamed: 243,Unnamed: 244,Unnamed: 245,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,01DSRPSZTDW2PGK1YWYXJGKZZ0,FILA,400010319073,Original Fitness Sneakers,Vintage Fitness leather sneakers with logo pri...,TheMensStore/Shoes/Sneakers/LowTop,2019-11-15 23:36:38.98161+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/fila-original-...,Leather/synthetic upper\nLace-up closure\nText...,"{""Needs Attributes""}","'design':12 'fila':1A 'fit':3A,6 'leather':7 '...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01DSQXJBX0R7DCW7KTAC1SW547,CHANEL,400011497371,HAT,NaN,Unknown,2019-11-15 16:15:34.809725+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/chanel-hat/pro...,WOOL TWEED & FELT,"{""Needs Review""}",'chanel':1A 'hat':2A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01DPGV8TGRAB993PF7Z3YWG2VR,Frame,LWBT0099,Petit Oval Buckle Belt,A Timeless Leather Belt Crafted From Smooth Co...,Accessories,2019-10-06 15:33:38.451504+00,2019-12-19 20:40:30.786144+00,NaN,https://frame-store.com/products/petit-oval-bu...,NaN,{},"'belt':5A,9 'buckl':4A,21 'cowhid':13 'craft':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01DSR8G3F7DBRTMP8THF97XSQ2,Lilly Pulitzer Kids,400011844797,Little Gir's & Girl's Ariana One-Piece UPF 50+...,Pretty ruffle sleeves and trim elevate essenti...,"JustKids/Girls214/Girls/SwimwearCoverups,JustK...",2019-11-15 19:26:34.947426+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/lilly-pulitzer...,Scoopneck\nAdjustable straps\nFlutter sleeves\...,"{""Needs Review""}",'50':14A 'allov':28 'ariana':9A 'color':27 'el...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01DSR8G5GP519DEDCSKBMWQVK5,Kissy Kissy,400094474585,Baby Girl's Endearing Elephants Pima Cotton Co...,Versatile convertible gown with elephant applique,JustKids/Baby024months/InfantGirls/FootiesRompers,2019-11-15 19:26:36.995998+00,2019-12-19 20:40:30.786144+00,NaN,https://www.saksfifthavenue.com/kissy-kissy-ba...,V-neckline\nLong sleeves\nFront snap closure\n...,"{""Needs Review""}","'appliqu':17 'babi':3A 'convert':10A,13 'cotto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [11]:
# perform inner join
combined = pd.merge(outfit, full_data[['product_id', 'description']], on='product_id')
combined

,outfit_id,product_id,outfit_item_type,brand,product_full_name,description
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt,A nice skirt
1,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt,A nice skirt
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank,A nice tank
3,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank,A nice tank
4,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory,kate spade new york,medium margaux leather satchel,A nice bag
...,...,...,...,...,...,...
4776,01E6M4W99KF80RD4N7FFCW87SM,01DS43PDN9P4XDPFSED3ZSC8N5,bottom,Vince,Belted Crepe Wide-Leg Pants,"Tailored in a leg-lengthening silhouette, Vinc..."
4777,01E6M4W99M1NSW0CJS736M1EJ6,01DS43PDN9P4XDPFSED3ZSC8N5,bottom,Vince,Belted Crepe Wide-Leg Pants,"Tailored in a leg-lengthening silhouette, Vinc..."
4778,01E6M4W99M8GRP5ZXJ3FT1903G,01DS43PDN9P4XDPFSED3ZSC8N5,bottom,Vince,Belted Crepe Wide-Leg Pants,"Tailored in a leg-lengthening silhouette, Vinc..."
4779,01E6M4W99MK8AHY989Y9Z99AQ8,01DS43PDN9P4XDPFSED3ZSC8N5,bottom,Vince,Belted Crepe Wide-Leg Pants,"Tailored in a leg-lengthening silhouette, Vinc..."


In [12]:
combined.isna().sum()

outfit_id              0
product_id             0
outfit_item_type       0
brand                  0
product_full_name      0
description          141
dtype: int64

In [13]:
combined.fillna('', inplace=True)

In [14]:
# lower-case the columns
col = ['outfit_item_type', 'brand', 'product_full_name', 'description'] 
for columns in col:
    combined[columns] = combined[columns].str.lower() 
combined.sort_values(by='outfit_id', inplace=True)
combined.reset_index(drop=True, inplace=True)

In [15]:
# define stopwords
stopwords_gensim = list(STOPWORDS)
stopwords_NLTK = list(stopwords.words("english"))
stopwords_combined = list(set(stopwords_gensim + stopwords_NLTK))
negatives = ['not','nor','no','neither', 'never', 'bottom', 'top'] # took out the negative words for a more accurate analysis
stopwords_combined = list(filter(lambda x: x not in negatives, stopwords_combined))
stopwords_combined.sort()
stopwords_expression = '|'.join(stopwords_combined)
stopwords_pattern = f'({stopwords_expression})'

In [16]:
# define stemming function
def stem_text(text):
    porter = PorterStemmer()
    tokens = text.split()
    stemmed_tokens = [porter.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

In [17]:
# combine features
combined['text_combined'] = combined['brand'] + ' ' + combined['product_full_name'] + ' ' + combined['description']

In [18]:
# clean, stem and remove stopwords from the text_combined
combined['text_combined'] = combined['text_combined'].astype(str)
combined['text_combined'] = combined['text_combined'].str.replace(r'[^\w\s]',' ')
combined['text_combined'] = combined['text_combined'].str.replace(rf'\b{stopwords_pattern}\b','')
combined['text_combined'] = combined['text_combined'].apply(stem_text)

In [19]:
combined.head()

,outfit_id,product_id,outfit_item_type,brand,product_full_name,description,text_combined
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,eileen fisher,slim knit skirt,a nice skirt,eileen fisher slim knit skirt nice skirt
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,eileen fisher,rib mock neck tank,a nice tank,eileen fisher rib mock neck tank nice tank
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory,kate spade new york,medium margaux leather satchel,a nice bag,kate spade new york medium margaux leather sat...
3,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2ZFDYRYY5TRQZJTBD,shoe,tory burch,penelope mid cap toe pump,a nice shoe,tori burch penelop mid cap toe pump nice shoe
4,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,eileen fisher,slim knit skirt,a nice skirt,eileen fisher slim knit skirt nice skirt


In [20]:
# load medium-sized corpus of English documents from spacy
nlp = en_core_web_md.load()

In [21]:
# perform vectorization of text_combined using pre-trained word2vec embeddings
description = combined['text_combined'].astype(str).tolist()
description_vectors = []
for i in description:
    temp_description = nlp(i)
    description_vectors.append(temp_description.vector)
combined['doc2vec'] = description_vectors 

In [24]:
def outfit_recommendation(outfit_type:str(), input_value:str(), isproductid=True):
    '''
    This function takes as an input three arguments:
    1) outfit_type: string; indicates type of the outfit that a user is looking for 
    (should be one of the following: ['bottom','top','accessory','shoe','onepiece'])
    2) input_value: string; represents the string (either product ID or free form text such as brand and description)
    for which the recommendations are returned
    3) isproductid: boolean; if True, input_value is considered a product ID; 
    else, input_value is treated as a free form text. By default is equal to 'True'
    
    For an input outfit, the function returns a dataframe with the product recommendations
    '''
    
    if isproductid: 
        # match info for the input product ID if there is an exact match
        if input_value in combined.product_id.unique():
            output = combined[combined.product_id==input_value]
        
        # find the closest product ID using fuzzy matching if the input product ID is misspelled/not found in data
        else:
            highestfuzz = process.extractOne(input_value, combined.product_id.values)
            output = combined[combined.product_id==highestfuzz[0]]

        print('The product with the closest product ID:\n')
        display(output.head(1).drop(columns=['text_combined','doc2vec']))
    
    else: 
        # clean, stem and remove stopwords from the input text
        input_clean = input_value.lower() 
        input_clean= re.sub(r'[^\w\s]', ' ', input_clean)
        input_clean = re.sub(rf'\b{stopwords_pattern}\b', '', input_clean)
        input_clean = stem_text(input_clean)

        # perform vectorization of the cleaned input and create an embedding matrix
        temp_input_clean = nlp(input_clean)
        temp_vectors = combined.loc[combined.outfit_item_type==outfit_type, 'doc2vec'].tolist()
        temp_text = combined.loc[combined.outfit_item_type==outfit_type, 'text_combined'].tolist()
        temp_vectors.append(temp_input_clean.vector)
        temp_text.append(input_clean)
        vector_df = pd.DataFrame(temp_vectors)
        vector_df.index = temp_text

        # calculate cosine similarity between two vectors
        similarities = pd.DataFrame(cosine_similarity(vector_df.values), columns=temp_text, index=temp_text)

        # find most similar vector to the input and retrieve the matched item
        top_similarities = similarities.unstack().reset_index().rename(columns={0:'score'})
        top_similarities = top_similarities.sort_values(by='score', ascending=False)
        top_similarities = top_similarities[top_similarities['score'] < 0.99999]
        match = top_similarities.loc[top_similarities['level_0']==input_clean, :].reset_index(drop = True)
        match_item = match.loc[0,'level_1']
        output = combined[combined.text_combined==match_item]

        print('The product with the closest matched description:\n')
        display(output.drop(columns=['text_combined','doc2vec']))
    
    # find all outfit combinations belonging to the same outfit ID 
    print('\nRecommendations for the identified product:')
    for id in output.outfit_id:
        print('\n')
        display(outfit[(outfit.outfit_id == id)&(outfit.product_id != output['product_id'].iloc[0])])

## Example 1 (Product ID, Exact Match)

In [25]:
outfit_recommendation('shoe', '01DMBRYVA2ZFDYRYY5TRQZJTBD')

The product with the closest product ID:



,outfit_id,product_id,outfit_item_type,brand,product_full_name,description
3,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2ZFDYRYY5TRQZJTBD,shoe,tory burch,penelope mid cap toe pump,a nice shoe



Recommendations for the identified product:




,outfit_id,product_id,outfit_item_type,brand,product_full_name
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory,kate spade new york,medium margaux leather satchel


,outfit_id,product_id,outfit_item_type,brand,product_full_name
4,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
5,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank
7,01DMHCX50CFX5YNG99F3Y65GQW,01DMHCNT41E14QWP503V7CT9G6,accessory,Nina,Crystal Clutch


,outfit_id,product_id,outfit_item_type,brand,product_full_name
8,01DMHRX35M2DPVYVQ1PNER4S4B,01DMBRYVA2Q2ST7MNYR6EEY4TK,onepiece,Equipment,Chemelle Midi Dress
10,01DMHRX35M2DPVYVQ1PNER4S4B,01DMHCNT41E14QWP503V7CT9G6,accessory,Nina,Crystal Clutch


,outfit_id,product_id,outfit_item_type,brand,product_full_name
11,01DMHRYZDTF2NBJ13ZEARKHA3T,01DMBRYVA2Q2ST7MNYR6EEY4TK,onepiece,Equipment,Chemelle Midi Dress
12,01DMHRYZDTF2NBJ13ZEARKHA3T,01DMBRYVA2S5T9W793F4CY41HE,accessory,kate spade new york,medium margaux leather satchel


## Example 2 (Product ID, Mismatch)

In [26]:
outfit_recommendation('shoe', '01DMBRYVA2ZFDYRYY5TRQZJTBA')

The product with the closest product ID:



,outfit_id,product_id,outfit_item_type,brand,product_full_name,description
3,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2ZFDYRYY5TRQZJTBD,shoe,tory burch,penelope mid cap toe pump,a nice shoe



Recommendations for the identified product:




,outfit_id,product_id,outfit_item_type,brand,product_full_name
0,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
1,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank
2,01DDBHC62ES5K80P0KYJ56AM2T,01DMBRYVA2S5T9W793F4CY41HE,accessory,kate spade new york,medium margaux leather satchel


,outfit_id,product_id,outfit_item_type,brand,product_full_name
4,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2P5H24WK0HTK4R0A1,bottom,Eileen Fisher,Slim Knit Skirt
5,01DMHCX50CFX5YNG99F3Y65GQW,01DMBRYVA2PEPWFTT7RMP5AA1T,top,Eileen Fisher,Rib Mock Neck Tank
7,01DMHCX50CFX5YNG99F3Y65GQW,01DMHCNT41E14QWP503V7CT9G6,accessory,Nina,Crystal Clutch


,outfit_id,product_id,outfit_item_type,brand,product_full_name
8,01DMHRX35M2DPVYVQ1PNER4S4B,01DMBRYVA2Q2ST7MNYR6EEY4TK,onepiece,Equipment,Chemelle Midi Dress
10,01DMHRX35M2DPVYVQ1PNER4S4B,01DMHCNT41E14QWP503V7CT9G6,accessory,Nina,Crystal Clutch


,outfit_id,product_id,outfit_item_type,brand,product_full_name
11,01DMHRYZDTF2NBJ13ZEARKHA3T,01DMBRYVA2Q2ST7MNYR6EEY4TK,onepiece,Equipment,Chemelle Midi Dress
12,01DMHRYZDTF2NBJ13ZEARKHA3T,01DMBRYVA2S5T9W793F4CY41HE,accessory,kate spade new york,medium margaux leather satchel


## Examples 3 and 4 (Product Description)

In [27]:
outfit_recommendation('bottom', 'Sexy silky, a-line mini skirt zipper Benson skirt', isproductid=False)

The product with the closest matched description:



,outfit_id,product_id,outfit_item_type,brand,product_full_name,description
14,01DQ63P636Q4BQVCKT6Z4S41G5,01DPKMGJ33SDFXM7XHGPQJWQ12,bottom,reformation,benson skirt,sexy silky. this is an a-line mini skirt with ...



Recommendations for the identified product:




,outfit_id,product_id,outfit_item_type,brand,product_full_name
14,01DQ63P636Q4BQVCKT6Z4S41G5,01DPCRZWX4S2Z8Q5HYDFM4HNEG,shoe,J.Crew,Pointed-toe flats in suede
15,01DQ63P636Q4BQVCKT6Z4S41G5,01DPET2NWSA221STZF740BZ9SW,top,Veronica Beard,Ashlynn Blouse


In [28]:
outfit_recommendation('bottom', 'slim fitting, straight leg pant with a center back zipper and slightly cropped leg', isproductid=False)

The product with the closest matched description:



,outfit_id,product_id,outfit_item_type,brand,product_full_name,description
26,01DQ8ME3M3QS9MQGZCQHXDHE1R,01DPKMH0D252JKMAA27MFCT5GM,bottom,reformation,marlon pant,let your pants do the talking. this is a slim ...



Recommendations for the identified product:




,outfit_id,product_id,outfit_item_type,brand,product_full_name
24,01DQ8ME3M3QS9MQGZCQHXDHE1R,01DPEHS0XH9PDD1GH5ZE4P43A2,accessory,Sole Society,Cassi Belt Bag
26,01DQ8ME3M3QS9MQGZCQHXDHE1R,01DPKN20Q3J0BE3CS896DQB6ER,top,Reformation,Jane Sweater
27,01DQ8ME3M3QS9MQGZCQHXDHE1R,01DPKNHQDG6GPTKV97CFQRJDHE,shoe,Reformation,Giulia Satin Heel


In [29]:
print(datetime.now()-start)

0:04:58.494088
